# Create test and train splits.

## Create Corpus

In [ ]:
import pandas as pd
from copy import copy
import math, re
from nltk import word_tokenize
import random as rd
import csv
from ast import literal_eval
import matplotlib.pyplot as plt
import numpy as np






data_complete = "../data/amazon.csv"
data_filtered = "../data/amazon_sample.csv"

## Describe Dataset

In [ ]:
df = pd.read_csv(data_filtered)
print(data_filtered, df.columns)

star1 = np.array([df.star_rating.value_counts()[1], 0])
star2 = np.array([df.star_rating.value_counts()[2], 0])
star3 = np.array([0, df.star_rating.value_counts()[3]])
star4 = np.array([0, df.star_rating.value_counts()[4]])
star5 = np.array([0, df.star_rating.value_counts()[5]])

In [ ]:
%matplotlib inline
df.star_rating.value_counts()

print(star1, star2, star3, star4, star5)
width=0.9
plt.figure(figsize=(5,5))
p1 = plt.bar([0,1], star1, width, color="darkred")
p2 = plt.bar([0,1], star2, width, bottom=star1, color="red")
p3 = plt.bar([0,1], star3, width, bottom=star1+star2, color="lightgreen")
p4 = plt.bar([0,1], star4, width, bottom=star1+star2+star3, color="green")
p5 = plt.bar([0,1], star5, width, bottom=star1+star2+star3+star4, color="darkgreen")




plt.ylabel('Number of product reviews')
plt.xlabel('Binary rating')

plt.title('Distribution of skalar-rating (star-rating)')

plt.xticks([0,1],('negative', 'positive'))

plt.legend(('1 Star', '2 Stars', '3 Stars', '4 Stars', '5 Stars'))

#plt.show()
plt.savefig("../figs/rating-distribution.png")

In [ ]:
df.bool_rating.value_counts()

## Create Dataset

In [ ]:
n_product = [ 50, 50, 201, 169, 100, 100, 100]
n_reviews = [ 110+29, 979+411, 842+173, 1410+251, 578+120,210+7, 719+81]
df = pd.read_csv(data_filtered)
print(len(df))
df = df.drop_duplicates(subset=["review_body"])
print(len(df))
print(df.columns)

In [ ]:
import operator
df = pd.read_csv(data_filtered)
print(len(df))
df = df.drop_duplicates(subset=["review_body"])
print(len(df))


df["test_idx"] = -1
categorie_keys = df['product_category'].value_counts(dropna=False).keys().tolist()
categorie_counts = df['product_category'].value_counts(dropna=False).tolist()
categorie_count_dict = dict(zip(categorie_keys, categorie_counts))
four_largest_categories = list(sorted(categorie_count_dict.items(), key=operator.itemgetter(1), reverse=True))[:4]

dfs = []
for rit in range(0, 10):
    sets = []

    for category , _ in four_largest_categories:
        df_category = df[df.product_category == category]
        rid_pos = df_category[df_category.bool_rating == True].review_id.to_list()
        rid_neg = df_category[df_category.bool_rating == False].review_id.to_list()
        rid_neg = rid_neg if len(rid_neg) < 56 else rd.sample(rid_neg, 56)
        rid_pos = rid_pos if len(rid_pos) < 56 else rd.sample(rid_pos, 56)
        #print(len(rid_pos), len(rid_neg))
        #print("unique", len(list(set(rid_pos))), len(list(set(rid_neg))))
        sets = sets + rid_pos + rid_neg
         
        df_product = df[df.review_id.isin(sets)]
        df_product["test_idx"] = rit
        dfs.append(df_product)

df_test_2 = pd.concat(dfs)
df_test_2.to_csv("../data/df_test_2.csv")

print(df_test_2.columns)
df_test_2.describe()



In [ ]:
df = pd.read_csv(data_filtered)
df.tokens = df.title_tokenized
print(len(df))
df = df.drop_duplicates(subset=["review_headline"])
print(len(df))


df["test_idx"] = -1
categorie_keys = df['product_category'].value_counts(dropna=False).keys().tolist()
categorie_counts = df['product_category'].value_counts(dropna=False).tolist()
categorie_count_dict = dict(zip(categorie_keys, categorie_counts))
four_largest_categories = list(sorted(categorie_count_dict.items(), key=operator.itemgetter(1), reverse=True))[:4]

dfs = []
for rit in range(0, 10):
    sets = []

    for category , _ in four_largest_categories:
        df_category = df[df.product_category == category]
        rid_pos = df_category[df_category.bool_rating == True].review_id.to_list()
        rid_neg = df_category[df_category.bool_rating == False].review_id.to_list()
        rid_neg = rid_neg if len(rid_neg) < 56 else rd.sample(rid_neg, 56)
        rid_pos = rid_pos if len(rid_pos) < 56 else rd.sample(rid_pos, 56)
        #print(len(rid_pos), len(rid_neg))
        #print("unique", len(list(set(rid_pos))), len(list(set(rid_neg))))
        sets = sets + rid_pos + rid_neg
         
        df_product = df[df.review_id.isin(sets)]
        df_product["test_idx"] = rit
        dfs.append(df_product)

df_test_2 = pd.concat(dfs)
df_test_2.to_csv("../data/df_test_2_title.csv")

print(df_test_2.columns)
df_test_2.describe()





In [ ]:
import operator
import pandas as pd
df = pd.read_csv(data_filtered)
print(len(df))
df = df.drop_duplicates(subset=["review_body"])
print(len(df))

dfs = []
for itc, category in enumerate(df.product_category.unique()):
    df_category = pd.DataFrame()
    df_all_categorie =  df[df.product_category == category]
    product_ids = []
    print("======", itc, "===", category, "======")
    while len(df_category) < n_reviews[itc] or len(df_category) == len(df_all_categorie):
        product_id = rd.choice(df_all_categorie.product_id.to_list())
        if product_id in product_ids:
            continue

        df_product = df[df.product_id == product_id]
        df_category = pd.concat([df_category, df_product])
    print("category:", category, df_category.shape,  n_reviews[itc])
    df_category["test_idx"] = itc
    
    
    
    dfs.append(df_category)
    
df_test_1 = pd.concat(dfs)
print("ready, shape", df_test_1.shape)
print("test_1 set", df_test_1.test_idx.value_counts())
df_test_1.to_csv("../data/df_test_1.csv")

print(df_test_1.columns)
print(df_test_1.describe())

In [ ]:
df = pd.read_csv(data_filtered)
df.tokens = df.title_tokenized
print(len(df))
df = df.drop_duplicates(subset=["review_headline"])
print(len(df))


data_set_review_body = "data/set_review_headline.csv"
dfs = []
for it, category in enumerate(df.product_category.unique()):
    df_category = pd.DataFrame()
    df_all_categorie =  df[df.product_category == category]
    product_ids = []
    print("======", it, "===", category, "======")
    while len(df_category) < n_reviews[it] or len(df_category) == len(df_all_categorie):
        product_id = rd.choice(df_all_categorie.product_id.to_list())
        if product_id in product_ids:
            continue

        df_product = df[df.product_id == product_id]
        df_category = pd.concat([df_category, df_product])
        df_category["test_idx"] = itc


        print("category:", category, df_category.shape,  n_reviews[it])
    dfs.append(df_category)
    
df = pd.concat(dfs)
print("ready, shape", df.shape)

df["tokens"] = df["title_tokenized"]

df.to_csv("../data/df_test_1_title.csv")

print("ready")

In [ ]:
dfs[0]